<a href="https://colab.research.google.com/github/Pavun-KumarCH/Agentic-RAG-Systems/blob/main/Agentic_RAG_llama_index_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title requirements
%pip install -U llama-index llama-parse llama-hub
%pip install llama-index-vector-stores-redis
%pip install llama-index-storage-docstore-redis
%pip install llama-index-storage-chat-store-redis
%pip install llama-index-llms-cohere
%pip install llama-index-embeddings-cohere
%pip install llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


In [18]:
import os
from google.colab import userdata
# Replace with your actual keys
os.environ["LLAMA_CLOUD_API_KEY"] = userdata.get("LLAMA_CLOUD_API_KEY")
os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")

# Redis connection settings
REDIS_HOST = "localhost"
REDIS_PORT = 6379
REDIS_PASSWORD = ""  # Use if necessary

In [17]:
# need this for running llama-index code in Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

In [19]:
!mkdir -p 'data/'
!wget 'https://raw.githubusercontent.com/redis-developer/LLM-Document-Chat/main/docs/2022-chevrolet-colorado-ebrochure.pdf' -O 'data/2022-chevrolet-colorado-ebrochure.pdf'

--2024-10-16 20:51:09--  https://raw.githubusercontent.com/redis-developer/LLM-Document-Chat/main/docs/2022-chevrolet-colorado-ebrochure.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3566101 (3.4M) [application/octet-stream]
Saving to: ‘data/2022-chevrolet-colorado-ebrochure.pdf’

data/2022-chevrolet 100%[===================>]   3.40M  --.-KB/s    in 0.06s   

2024-10-16 20:51:10 (61.4 MB/s) - ‘data/2022-chevrolet-colorado-ebrochure.pdf’ saved [3566101/3566101]



In [20]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# Initialize the PDF parser
parser = LlamaParse(
    result_type="markdown"  # You can choose between "markdown" and "text"
)

# Specify file extractor for PDFs
file_extractor = {".pdf": parser}

# Load the document using SimpleDirectoryReader
reader = SimpleDirectoryReader("./data", file_extractor=file_extractor)
documents = reader.load_data()


20:51:23 httpx INFO   HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"
Started parsing the file under job_id 8d872349-8d73-460c-8f5b-39a6b4b7e824
20:51:24 httpx INFO   HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/8d872349-8d73-460c-8f5b-39a6b4b7e824 "HTTP/1.1 200 OK"
20:51:26 httpx INFO   HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/8d872349-8d73-460c-8f5b-39a6b4b7e824 "HTTP/1.1 200 OK"
20:51:29 httpx INFO   HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/8d872349-8d73-460c-8f5b-39a6b4b7e824 "HTTP/1.1 200 OK"
20:51:31 httpx INFO   HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/8d872349-8d73-460c-8f5b-39a6b4b7e824 "HTTP/1.1 200 OK"
20:51:33 httpx INFO   HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/8d872349-8d73-460c-8f5b-39a6b4b7e824 "HTTP/1.1 200 OK"
20:51:35 httpx INFO   HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/8d872349-8d7

In [23]:
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.core.ingestion import (
    DocstoreStrategy,
    IngestionPipeline,
    IngestionCache,
)
from llama_index.storage.kvstore.redis import RedisKVStore as RedisCache
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.redis import RedisVectorStore

from redisvl.schema import IndexSchema

# Ensure the API key is set
COHERE_API_KEY = os.getenv("COHERE_API_KEY")  # Fetch from environment variables

# Initialize the embedding model with the API key
embed_model = CohereEmbedding(input_type="search_document", api_key=COHERE_API_KEY)

custom_schema = IndexSchema.from_dict(
    {
        "index": {
            "name": "chevy-colorado",
            "prefix": "pdf:chunk",
            "key_separator": ":"
          },
        # Customize fields that are indexed
        "fields": [
            # Required fields for LlamaIndex
            {"type": "tag", "name": "id"},
            {"type": "tag", "name": "doc_id"},
            {"type": "text", "name": "text"},
            # Custom vector field for Cohere embeddings
            {
                "type": "vector",
                "name": "vector",
                "attrs": {
                    "dims": 1024,
                    "algorithm": "hnsw",
                    "distance_metric": "cosine",
                },
            },
        ],
    }
)

In [24]:
vector_index_pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),  # Split the documents into sentences
        embed_model,  # Embed the split sentences using Cohere embeddings
    ],
    # Redis document store to store the original documents
    docstore=RedisDocumentStore.from_host_and_port(
        REDIS_HOST, REDIS_PORT, namespace="doc-store"
    ),
    # Redis vector store for storing vectors (from embeddings) in Redis
    vector_store=RedisVectorStore(
        schema=custom_schema,  # The custom schema defined earlier
        redis_url=f"redis://{REDIS_HOST}:{REDIS_PORT}",
    ),
    # Cache to avoid reprocessing documents
    cache=IngestionCache(
        cache=RedisCache.from_host_and_port(REDIS_HOST, REDIS_PORT),
        collection="doc-cache",  # A namespace for caching document chunks
    ),
    # Upsert strategy ensures documents are updated if they already exist
    docstore_strategy=DocstoreStrategy.UPSERTS,
)


In [25]:
vector_index_pipeline.run(documents=documents, show_progress=True)

Parsing nodes:   0%|          | 0/24 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/28 [00:00<?, ?it/s]

20:56:24 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
20:56:24 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
20:56:24 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
20:56:24 llama_index.vector_stores.redis.base INFO   Added 28 documents to index chevy-colorado


[TextNode(id_='28b2200b-aae4-47f7-a665-03a502ee9e8b', embedding=[-0.043304443, -0.027618408, -0.030380249, 0.008377075, -0.055480957, -0.010276794, -0.03933716, -0.00052022934, -0.043914795, -0.009002686, 0.019180298, 0.021942139, 0.01436615, -0.050750732, 0.06451416, -0.02645874, 0.049194336, -0.0010442734, 0.022827148, -0.0029067993, -0.012397766, 0.04736328, -0.011291504, 0.027435303, 0.00349617, 0.03250122, -0.028671265, -0.016555786, -0.010223389, -0.014045715, -0.011856079, 0.034240723, 0.014961243, 0.03250122, 0.0029182434, 0.09075928, 0.023101807, 0.01449585, 0.033843994, 0.02406311, 0.021316528, -0.010696411, 0.015083313, 0.051849365, 0.0057258606, -0.052246094, 0.03491211, -0.04550171, 0.023773193, 0.011177063, 0.038726807, 0.04171753, 0.03829956, 0.03326416, -0.014953613, 0.019119263, 0.02558899, 0.07318115, 0.021469116, -0.08782959, -0.052093506, 0.048339844, 0.033569336, 0.05026245, -0.010681152, -0.010917664, -0.046081543, 0.013282776, 0.0014381409, -0.0075912476, 0.01612

In [26]:
vector_index_pipeline.run(documents=documents)

[]

# Building the ReAct Agent
In this section we define a ReAct agent that will perform RAG over a PDF document using the Cohere command-r-plus language model.

**ReAct** is an agent based on a query engine over your data. For each chat interaction, the agent enter a ReAct loop:

decide whether to use the query engine tool and come up with appropriate input
(optional) use the query engine tool and observe its output
decide whether to repeat or give final response
## **Agent Setup**
Below we define our LLM, embedding model, and a chat memory layer backed by Redis for conversation history and context.

We define both a vector index (for semantic search) and summary index (for summarization) for the document -- used as tools by the agent.

In [29]:
# Setup Cohere as the base embedding model and LLM
from llama_index.llms.cohere import Cohere
from llama_index.core import Settings

# Make sure to set your API keys before this step
COHERE_API_KEY = os.getenv("COHERE_API_KEY")  # Get the API key from environment variables

# Initialize the Cohere embedding model with the API key
embed_model = CohereEmbedding(input_type="search_query", api_key=COHERE_API_KEY)

# Initialize the LLM with the desired model
llm = Cohere(model="command-r-plus")

# Set the embedding and LLM settings
Settings.llm = llm
Settings.embed_model = embed_model



In [30]:
# Set up memory for the Agent
from llama_index.storage.chat_store.redis import RedisChatStore
from llama_index.core.memory import ChatMemoryBuffer

# Build chat memory backed by Redis
chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,
    chat_store=RedisChatStore(redis_url=f"redis://{REDIS_HOST}:{REDIS_PORT}", ttl=300),
    chat_store_key="user_1"
)

In [31]:
from llama_index.core.agent import ReActAgent
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.node_parser import SentenceSplitter

import pickle


async def build_doc_agent(doc):
    # ID will be base + parent
    file_name = doc.metadata["file_name"]
    file_path = f"./data/{file_name}"
    file_id = file_name.replace("-", "_").strip(".pdf")
    print(file_id)

    # Run ingestion
    vector_index_pipeline.run(documents=[doc], show_progress=True)

    # Grab the nodes
    node_parser = SentenceSplitter()
    nodes = node_parser.get_nodes_from_documents([doc])

    # Define vector index and query engine
    vector_index = VectorStoreIndex.from_vector_store(
        vector_index_pipeline.vector_store
    )
    vector_query_engine = vector_index.as_query_engine()

    # Build summary index and extract a summary
    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize"
    )
    summary_out_path = f"./data/{file_name}_summary.pkl"
    summary = str(
        await summary_query_engine.aquery(
            "Extract a concise 1-2 line summary of this document"
        )
    )
    pickle.dump(summary, open(summary_out_path, "wb"))

    # Define agent tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name=f"vector_tool_{file_id}",
                description=f"Useful for questions related to specific facts about the chevy colorado",
            ),
        )
    ]

    # Build ReAct agent
    agent = ReActAgent.from_tools(
        query_engine_tools,
        llm=llm,
        verbose=True,
        memory=chat_memory,
        context=f"""\
You are a specialized, trustworthy, helpful, and technical customer support agent designed to answer queries about the Chevy Colorado 2022 vehicle.
Use the available tools provided when answering a question. Do NOT just blindly make things up about the car unless it is grounded by the retrieved sources.\
""")

    return agent, summary

In [33]:
agent, doc_summary = await build_doc_agent(documents[0])

2022_chevrolet_colorado_ebrochure
20:59:37 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
20:59:37 llama_index.core.agent.react.formatter WARNING   ReActChatFormatter.from_context is deprecated, please use `from_defaults` instead.


In [34]:
doc_summary

'A file path for a 2022 Chevrolet Colorado ebrochure.'

In [35]:
response = agent.chat("What is the seating capacity of the vehicle?")
print(str(response))

> Running step 7147f4cb-d98a-4555-bbb4-bde38ea3c874. Step input: What is the seating capacity of the vehicle?
20:59:51 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_2022_chevrolet_colorado_ebrochure
Action Input: {'input': 'seating capacity'}
20:59:51 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
20:59:51 llama_index.vector_stores.redis.base INFO   Querying index chevy-colorado with query *=>[KNN 2 @vector $vector AS vector_distance] RETURN 5 id doc_id text _node_content vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 2
20:59:51 llama_index.vector_stores.redis.base INFO   Found 2 results for query with id ['pdf:chunk:fc5af5ae-94c0-4c24-a28a-84e496ccfd91', 'pdf:chunk:2318df3e-0f72-4d71-ad86-bfa68638b60f']
20:59:52 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "H

In [36]:
response = agent.chat("What is the towing capacity?")
print(str(response))

> Running step cab44fd1-46db-477b-9090-abe6c92907b4. Step input: What is the towing capacity?
21:00:11 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_2022_chevrolet_colorado_ebrochure
Action Input: {'input': 'towing capacity'}
21:00:11 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
21:00:11 llama_index.vector_stores.redis.base INFO   Querying index chevy-colorado with query *=>[KNN 2 @vector $vector AS vector_distance] RETURN 5 id doc_id text _node_content vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 2
21:00:11 llama_index.vector_stores.redis.base INFO   Found 2 results for query with id ['pdf:chunk:2318df3e-0f72-4d71-ad86-bfa68638b60f', 'pdf:chunk:fc5af5ae-94c0-4c24-a28a-84e496ccfd91']
21:00:14 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
O

In [37]:
response = agent.chat("Is there a trailer hitch on the back of the truck?")
print(str(response))

> Running step 835ca374-2d0d-4abe-99a6-e801908e62d9. Step input: Is there a trailer hitch on the back of the truck?
21:00:59 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_2022_chevrolet_colorado_ebrochure
Action Input: {'input': 'trailer hitch'}
21:00:59 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
21:00:59 llama_index.vector_stores.redis.base INFO   Querying index chevy-colorado with query *=>[KNN 2 @vector $vector AS vector_distance] RETURN 5 id doc_id text _node_content vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 2
21:00:59 llama_index.vector_stores.redis.base INFO   Found 2 results for query with id ['pdf:chunk:6e082b28-1f39-4717-80bb-0a03fe93a13c', 'pdf:chunk:272e75b6-aa2e-493d-b321-238fb81c0213']
21:01:02 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat

In [38]:
response = agent.chat("Tell me about the pros and cons of this truck.")
print(str(response))

> Running step 8fde3dab-8274-4bae-abf2-1162256f3e1d. Step input: Tell me about the pros and cons of this truck.
21:01:24 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_2022_chevrolet_colorado_ebrochure
Action Input: {'properties': AttributedDict([('input', 'pros and cons'), ('title', 'Pros and Cons of the 2022 Chevy Colorado')]), 'required': ['input'], 'type': 'object'}
21:01:24 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
21:01:24 llama_index.vector_stores.redis.base INFO   Querying index chevy-colorado with query *=>[KNN 2 @vector $vector AS vector_distance] RETURN 5 id doc_id text _node_content vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 2
21:01:24 llama_index.vector_stores.redis.base INFO   Found 2 results for query with id ['pdf:chunk:272e75b6-aa2e-493d-b321-238fb81c02

In [39]:
agent.memory.chat_store.get_messages("user_1")

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the seating capacity of the vehicle?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Apologies, but I could not find any information about the seating capacity of the 2022 Chevy Colorado.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the towing capacity?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='The maximum trailering capacity varies depending on the model of the 2022 Chevy Colorado. For instance, the ZR2 model with a 3.6L V6 engine can tow up to 5,000 lbs.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Is there a trailer hitch on the back of the truck?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Yes, the 2022 Chevy Colorado has an available trailering package that includes a trailer hitch. This package enables the v

Incorporating Semantic Caching
We can also take advantage of frequently asked questions (live or prefetched) in order to improve response times.

In [40]:
from redisvl.extensions.llmcache import SemanticCache
from redisvl.utils.vectorize import HFTextVectorizer

emb = HFTextVectorizer(model="BAAI/bge-small-en-v1.5")

cache = SemanticCache(
    name="chevy_cache",
    prefix="cache",
    distance_threshold=0.2,
    ttl=60,
    vectorizer=emb
)

21:02:14 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: cuda
21:02:14 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [42]:
def invoke_agent(prompt: str) -> str:
    if cached_result := cache.check(prompt=prompt):
        response = cached_result[0]['response']
        return response
    response = agent.chat(prompt)
    # cache.store(prompt=prompt, response=response.response)
    return response.response

Now we can perform a simple test with our agent and semantic caching enabled.



In [43]:
invoke_agent("How many doors does the truck have?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

> Running step f21c7666-1bf0-459b-8288-599c2645a62f. Step input: How many doors does the truck have?
21:02:49 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
Thought: (Implicit) I can answer without any more tools!
Answer: The 2022 Chevy Colorado is typically offered in two body styles: an Extended Cab with two doors and a Crew Cab with four doors. The Extended Cab features a 2+2 seating configuration, providing additional cargo space, while the Crew Cab offers a more traditional 4-door design with ample room for passengers.


'The 2022 Chevy Colorado is typically offered in two body styles: an Extended Cab with two doors and a Crew Cab with four doors. The Extended Cab features a 2+2 seating configuration, providing additional cargo space, while the Crew Cab offers a more traditional 4-door design with ample room for passengers.'

In [44]:
invoke_agent("How many passenger doors are on the truck?")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

> Running step 4a4e6aee-bfda-4d82-b74d-6ee19535cdf7. Step input: How many passenger doors are on the truck?
21:02:52 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
Thought: (Implicit) I can answer without any more tools!
Answer: The number of passenger doors depends on the cab style you choose. If you opt for the Extended Cab, there is one passenger door on the driver's side, resulting in a total of two doors (including the driver's door). On the other hand, the Crew Cab features two passenger doors, one on each side, for a total of four doors. So, depending on your preference and needs, you can choose between two or four passenger doors with the 2022 Chevy Colorado.


"The number of passenger doors depends on the cab style you choose. If you opt for the Extended Cab, there is one passenger door on the driver's side, resulting in a total of two doors (including the driver's door). On the other hand, the Crew Cab features two passenger doors, one on each side, for a total of four doors. So, depending on your preference and needs, you can choose between two or four passenger doors with the 2022 Chevy Colorado."

# Extending Semantic Caching
There are a few options for working with semantic caching in a true production setting:

1. Extract FAQs from your Knowledge Base (pdfs...). Use an LLM to help! Or use human experts. Prefetch into the cache.
2. Carefully, extract FAQs from conversation history. Prefetch in batches into the cache each day or week.
3. Cache in realtime, which is tricky and requires some tuning. This should typically be done only at the USER level, not across the entire domain or application.
Stay tuned for more dedicated guides on semantic caching for RAG Agents and Redis.